# PLAN

- [x] Acquisition
    - [x] read the csv into a dataframe
- [ ] Preparation
    - [ ] no missing values
    - [x] drop columns that are not needed
    - [x] change case to lower case
    - [x] make sure everything has right dtype
    - [ ] normalize what needs to be normalized
    - [x] rename columns for clarification
- [ ] Exploration
    - [ ] answer ALL questions raised
        - [x] Which locations are the most frequent sites of SSO?
        - [x] Which location have the most volume of overflow?
        - [x] What are most common root causes of SSO?
        - [x] Where do the majority of overflow go?

    - [ ] visualize important findings
    - [ ] decide what TODO items to keep
- [ ] Modeling
    - [ ] predict 
- [ ] Delivery
    - [ ] report
    - [ ] prezi slides
    - [ ] website

# ENVIRONMENT

In [62]:
import os
import acquire_sso as acquire
import prepare_sso as prepare
import pandas as pd
import numpy as np

# data visualization 
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import statsmodels.api as sm

from datetime import timedelta, datetime
from pylab import rcParams

# to explode the DataFrames and avoid truncation
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from fbprophet import Prophet

import warnings
warnings.filterwarnings('ignore')

# ACQUIRE

#### _Let's read in the data from the csv file and take a peek at te first five records._

In [2]:
df = acquire.read_data('saws-sso.csv')

In [3]:
df.head()

,SSO_ID,INSPKEY,SERVNO,REPORTDATE,SPILL_ADDRESS,SPILL_ST_NAME,TOTAL_GAL,GALSRET,GAL,SPILL_START,SPILL_STOP,HRS,CAUSE,COMMENTS,ACTIONS,WATERSHED,UNITID,UNITID2,DISCHARGE_TO,DISCHARGE_ROUTE,COUNCIL_DISTRICT,FERGUSON,Month,Year,Week,EARZ_ZONE,Expr1029,PIPEDIAM,PIPELEN,PIPETYPE,INSTYEAR,DWNDPTH,UPSDPTH,Inches_No,RainFall_Less3,SPILL ADDRESS,SewerAssetExp,NUM_SPILLS_COMPKEY,NUM_SPILLS_24MOS,PREVSPILL_24MOS,UNITTYPE,ASSETTYPE,LASTCLND,ResponseTime,ResponseDTTM,Public Notice,TIMEINT,Root_Cause,STEPS_TO_PREVENT,SPILL_START_2,SPILL_STOP_2,HRS_2,GAL_2,SPILL_START_3,SPILL_STOP_3,HRS_3,GAL_3
0,6582,567722.0,NaN,3/10/19,3200,THOUSAND OAKS DR,2100,2100.0,2100.0,3/10/2019 1:16:00 PM,3/10/2019 2:40:00 PM,1.400000,Grease,Spill ContainedReturned to SystemArea Cleaned ...,CLEANED MAIN,SALADO CREEK,66918,66917,STREET,None,NaN,172A2,3,2019,11,0.0,NaN,8.0,16.55,PVC,1997.0,NaN,NaN,NaN,NaN,3200 THOUSAND OAKS DR,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,0.45,10-Mar-19,False,24.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
1,6583,567723.0,NaN,3/10/19,6804,S FLORES ST,80,0.0,80.0,3/10/2019 2:25:00 PM,3/10/2019 3:45:00 PM,1.333333,Grease,Spill ContainedArea Cleaned and Disinfected,CLEANED MAIN,DOS RIOS,24250,24193,STORMDRAIN,None,3.0,251A3,3,2019,11,0.0,NaN,8.0,157.00,PVC,1988.0,NaN,NaN,NaN,NaN,6804 S FLORES,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,1.08,10-Mar-19,False,120.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
2,6581,567714.0,NaN,3/9/19,215,AUDREY ALENE DR,79,0.0,10.0,3/9/2019 6:00:00 PM,3/9/2019 7:30:00 PM,1.500000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,DOS RIOS,2822,3351,ALLEY,None,1.0,190E4,3,2019,10,0.0,NaN,8.0,350.00,CP,1955.0,NaN,NaN,NaN,NaN,215 Audrey Alene Dr,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,1.00,09-Mar-19,False,24.0,NaN,NaN,03/10/2019 09:36,03/10/2019 10:45,1.15,69.0,NaN,NaN,0.0,0.0
3,6584,567713.0,NaN,3/9/19,3602,SE MILITARY DR,83,0.0,83.0,3/9/2019 3:37:00 PM,3/9/2019 5:00:00 PM,1.383333,Grease,Spill ContainedArea Cleaned and DisinfectedFlu...,NaN,SALADO CREEK,92804,92805,EASEMENT,None,3.0,252C3,3,2019,10,0.0,NaN,8.0,213.91,PVC,1983.0,NaN,NaN,NaN,NaN,3602 SE MILITARY DR,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,0.55,09-Mar-19,False,120.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
4,6580,567432.0,NaN,3/6/19,100,PANSY LN,75,0.0,75.0,3/6/2019 9:40:00 AM,3/6/2019 9:55:00 AM,0.250000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,SALADO CREEK,61141,49543,STREET,None,2.0,192A7,3,2019,10,0.0,NaN,12.0,291.90,CP,1952.0,NaN,NaN,NaN,NaN,100 PANSY LN,NaN,2,2.0,15-Dec-18,GRAVITY,Sewer Main,NaN,0.00,06-Mar-19,False,3.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0


In [4]:
df['UNITTYPE'].value_counts()

GRAVITY       2508
OUTFALL        222
SIPHON         144
ABOVE           83
LATERAL         57
SUBMERSED       51
FORCE           44
STANDARD        29
                 9
A/R              6
IN SIPHON        2
JUNCTION         2
TANKHOLDIN       2
PMPCENTSP        2
OUT SIPHON       2
SLUDGE           1
OTHER            1
PMPSUBM          1
Name: UNITTYPE, dtype: int64

In [5]:
df['ASSETTYPE'].value_counts()

Sewer Main               2925
Sewer Lift Station        108
Sewer Service Line         62
Sewer Manhole              34
Sewer Liftstation          30
Sewer Valve                 7
Sewer Node                  3
Sewer Pump                  3
Sewer Treatment Plant       3
Plant Equipment             2
BYPASS LINE                 1
Sewer Miscellaneous         1
Bypass                      1
Sewer Force Main            1
Name: ASSETTYPE, dtype: int64

In [6]:
df.WATERSHED.value_counts()

DOS RIOS             1572
SALADO CREEK          790
LEON CREEK            668
MEDIO CREEK           146
CCMA (Subscriber)       4
Salado Creek            1
Leon Creek              1
Dos Rios                1
Name: WATERSHED, dtype: int64

In [7]:
(df.WATERSHED == 'LEON CREEK').sum()

668

In [8]:
df[df.WATERSHED == 'Leon Creek']

,SSO_ID,INSPKEY,SERVNO,REPORTDATE,SPILL_ADDRESS,SPILL_ST_NAME,TOTAL_GAL,GALSRET,GAL,SPILL_START,SPILL_STOP,HRS,CAUSE,COMMENTS,ACTIONS,WATERSHED,UNITID,UNITID2,DISCHARGE_TO,DISCHARGE_ROUTE,COUNCIL_DISTRICT,FERGUSON,Month,Year,Week,EARZ_ZONE,Expr1029,PIPEDIAM,PIPELEN,PIPETYPE,INSTYEAR,DWNDPTH,UPSDPTH,Inches_No,RainFall_Less3,SPILL ADDRESS,SewerAssetExp,NUM_SPILLS_COMPKEY,NUM_SPILLS_24MOS,PREVSPILL_24MOS,UNITTYPE,ASSETTYPE,LASTCLND,ResponseTime,ResponseDTTM,Public Notice,TIMEINT,Root_Cause,STEPS_TO_PREVENT,SPILL_START_2,SPILL_STOP_2,HRS_2,GAL_2,SPILL_START_3,SPILL_STOP_3,HRS_3,GAL_3
1703,2795,162383.0,NaN,6/7/12,7523,OLD PEARSALL RD,124,124.0,124.0,6/7/2012 11:33:00 AM,6/7/2012 12:25:00 PM,0.866667,Grease,APPLIED HTH,CLEANED MAIN,Leon Creek,41296,41265,DRAINAGE CULVERT,None,4.0,679D4,6,2012,23,0.0,GREASE,8.0,400.0,PVC,1993.0,NaN,NaN,0.0,0.0,7523 OLD PEARSALL RD,NaN,2,NaN,NaN,GRAVITY,Sewer Main,NaN,NaN,NaN,False,NaN,GREASE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Work with geocoded_saws.csv to clean up zip codes and lat and long

In [9]:
df_geocoded = acquire.read_data('geocoded_saws.csv')

In [10]:
df_geocoded.sewershed.value_counts()

dos rios             1566
salado creek          789
leon creek            671
medio creek           153
ccma (subscriber)       4
Name: sewershed, dtype: int64

In [11]:
df_geocoded.Postal[df_geocoded.sewershed == 'leon creek'].value_counts()

78227    122
78242     99
78228     77
78240     54
78251     45
78238     43
78250     31
78229     29
78243     25
78249     24
78254     19
78257     17
78211     13
78252     11
78221     11
78245      9
78255      8
78253      6
78237      6
78256      6
78015      3
78230      3
78224      3
78264      2
78073      2
1          2
78023      1
Name: Postal, dtype: int64

In [12]:
wrong_zips = [78219, 78218, 78208, 78154, 78244, 77377]

In [13]:
temp = df_geocoded[['spill_street_address', 'Postal', 'sewershed']][df_geocoded['Postal'].isin(wrong_zips)]
temp.sort_values(by='sewershed')

,spill_street_address,Postal,sewershed
386,"1112 muncey, san antonio, tx",78208,dos rios
1323,"4000 ih 10 e, san antonio, tx",78219,dos rios
1324,"700 at and t center pkwy, san antonio, tx",78219,dos rios
1325,"3099 houston st e, san antonio, tx",78219,dos rios
1326,"4065 ih 10 e, san antonio, tx",78219,dos rios
401,"2535 ih 35 n, san antonio, tx",78208,dos rios
400,"2004 23rd st, san antonio, tx",78208,dos rios
398,"1300 ervin, san antonio, tx",78208,dos rios
397,"300 jim st, san antonio, tx",78208,dos rios
396,"1700 rogers ave, san antonio, tx",78208,dos rios


In [14]:
temp[temp['sewershed']=='leon creek']

,spill_street_address,Postal,sewershed


#### I changed the following data so that it no longer shows up as errors.
- 32	12014 canyon rock ln, san antonio, tx	77377
    - Closest is in Spring Branch, 78070, but that's north of SAWS area, removed city and changed zip to 1 and removed long and lat

- 257	10710 spirit roam, san antonio, tx	78154
    - Changed to 10710 spirit roam, san antonio, tx 78254
    - (long: -98.704260, lat: 29.525840

- 778	29621 elkhorn ridge, san antonio, tx	78218
    - Changed to 29621 elkhorn ridge, boerne, tx 78015
    - (long: -98.662880, lat: 29.734010)
- also changed 7707 lost creek gap, san antonio, tx
    - changed to 7707 lost creek gap, boerne, tx 78015
    - (long: -98.636370, lat: 29.682290)
- also changed 8219 mystic chase, san antonio, tx
    - changed to 8219 mystic chase, boerne, tx 78015
    - (long: -98.643900, lat: 29.702590)

- 796	8400 redwing dr, san antonio, tx	78219
    - There is no Redwing Drive in SA, closest is north of Medina Lake... ??? removed city and changed zip to 1 and removed long and lat

- 1097	4707 crystal hill, san antonio, tx	78244
    - Changed to 4707 crystal hill, san antonio, tx 78238
    - (long: -98.620500, lat: 29.479150)

- 1131	89 grey, san antonio, tx	78208
    - changed to 89 Grey Bluff, San Antonio, TX 78252
    - (long: -98.62897, lat: 29.343267)

In [15]:
df_geocoded.Postal[df_geocoded.sewershed == 'dos rios'].value_counts()

78201    169
78228    162
78223    156
78207    124
78212    123
78213     83
78221     77
78216     75
78230     74
78210     61
78209     60
78237     54
78214     41
78220     34
78211     34
78229     32
78202     28
78226     26
78203     26
78224     25
78225     22
78204     17
78208     16
78205     12
78249      9
78215      8
78240      6
78219      4
78231      4
78264      3
78257      1
Name: Postal, dtype: int64

In [16]:
wrong_for_dos_rios = [77701, 81120, 78232, 78236, 78251, 78252, 
                      78255, 78257, 78259, 78606, 78064]

In [17]:
temp = df_geocoded[['spill_street_address', 'Postal', 'sewershed']][df_geocoded['Postal'].isin(wrong_for_dos_rios)]
temp.sort_values(by=['sewershed', 'Postal'])

,spill_street_address,Postal,sewershed
3086,"17203 nw military hwy, san antonio, tx",78257,dos rios
2946,"4315 rogers rd, san antonio, tx",78251,leon creek
2947,"9765 potranco rd, san antonio, tx",78251,leon creek
2948,"2310 elva forest, san antonio, tx",78251,leon creek
2949,"9926 sable arrow, san antonio, tx",78251,leon creek
2950,"8985 grissom rd, san antonio, tx",78251,leon creek
2951,"9765 potranco rd, san antonio, tx",78251,leon creek
2952,"9765 potranco rd, san antonio, tx",78251,leon creek
2953,"3302 timber view dr, san antonio, tx",78251,leon creek
2954,"1900 ashprington, san antonio, tx",78251,leon creek


In [18]:
temp[temp['sewershed']=='dos rios'].sort_values(by='Postal')

,spill_street_address,Postal,sewershed
3086,"17203 nw military hwy, san antonio, tx",78257,dos rios


I changed the following data so that it no longer shows up as errors.
Except that one address really is at the top of the dos rios sewershed.

- 2600 cima, san antonio, tx	77701	dos rios
    - changed to 2600 cima st, san antonio, tx 78228
    - (long: -98.54377, lat: 29.44466)
    
NOT SURE IF THE NEXT TWO LISTSINGS ARE CORRECT...
- 7500 us hwy 281 n, san antonio, tx	78064	dos rios
    - changed to 7500 us hwy 281 n, san antonio, tx	78216
    - (long: -98.48198, lat: 29.48261)
    
- 7900 us hwy 281 n, san antonio, tx	78232	dos rios
    - changed to 7900 us hwy 281 n, san antonio, tx	78216
    - (long: -98.482997, lat: 29.48876)
    
    
- 100 chappie james way, san antonio, tx	78236	dos rios
    - 100 chappie james way, san antonio, tx	78243
    - (long: -98.59991, lat: 29.3869)
    - also changed 108 chappie james way, san antonio 78243
    - (long: -98.59896, lat: 29.38401)
    
- 105 dillon walk, san antonio, tx	78251	dos rios
    - there is no dillon walk...should this be dillon road???
    - changed to 105 dillon road, san antonio, tx 78002
    - (long: -98.6987, lat:	29.31755)
    
- 12519 switchgrass, san antonio, tx	78252	dos rios
    - looks like correct zip code, but wrong sewershed
    - changed sewershed to medio creek
    - (long: -98.738300, lat: 29.360230)
    
- 6700 mountain top, san antonio, tx	78255	dos rios
    - looks like correct zip code, but wrong sewershed
    - changed sewershed to leon creek
    - (long: -98.660072, lat: 29.688419)
    
- 17203 nw military hwy, san antonio, tx	78257	dos rios
    - all looks correct... top of dos rios sewershed
    
- 800 hanover, san antonio, tx	78259	dos rios
    - changed to 800 hanover, san antonio, tx	78228
    - (long: -98.557060, lat: 29.448790)
    
- 100 pyle, san antonio, tx	78606	dos rios
    - changed to 100 pyle st, san antonio, tx	78223
    - (long: -98.445580, lat: 29.365900)
    
- 500 st agatha, san antonio, tx	81120	dos rios
    - changed to 500 st agatha, san antonio, tx	78207
    - (long: -98.532380, lat: 29.415290)
    

#### Other errors detected through Tableau...
# need to be corrected in csv file

Postal	Sewershed	Spill Street Address	X	Y

Should be medio creek:
- 78245	dos rios	2007 ares cv, san antonio, tx	-98.78637663	29.41156555
- 78245	dos rios	12503 ranch summit, san antonio, tx	-98.74192235	29.42448298
- 78245	dos rios	2380 grosenbacher rd, san antonio, tx	-98.72656996	29.40217192
- 78245	dos rios	2553 struck silver, san antonio, tx	-98.71700874	29.40479113



Should be dos rios:
- 78225	salado creek	4000 ih 35 n, san antonio, tx	-98.512	29.38424129
    - changed sewershed to dos rios
    - changed address to 4000 ih 35 n, san antonio, tx 78214
    - (long: -98.511903, lat: 29.382335)



Should be leon creek:
- 78228	salado creek	83 arrowhead dr, san antonio, tx	-98.57819317	29.47707174
    - changed sewershed to leon creek
    - (long: -98.573485, lat: 29.482164)





Changed to leon creek to match sewershed of others on this street:
- 10/24/18 0:00	2448400	0	2448400	10/24/18 13:50	10/28/18 13:29	95.65	i/i		monitored area	leon creek	1210	42319	creek bed - spilled into leon creek	leon creek		2.28E+08	10	2018	43	0	RAIN EVENT	54	1335	rcp	1965	0.79	0.49	10	9	outfall	sewer main		TRUE	RAIN EVENT	Capacity Project, 			0	0			0	0	108 chappie james way, san antonio, tx	78243	122	-98.59896	29.38401


Changed zip code:
- 78233	dos rios	2535 ih 35 n, san antonio, tx	-98.36316044	29.54880533
    - changed to 2535 ih 35 n, san antonio, tx 78208
    - (long: -98.452467, lat: 29.439920)
- 78233 dos rios  2623 ih 35 n, san antonio, tx -98.36316044	29.54880533
    - changed to 2623 ih 35 n, san antonio, tx 78211
    - (long: -98.450922, lat: 29.439939)
    
- 78237 salado creek 132 afton oaks e, san antonio, tx
    - changed to 132 afton oaks e, san antonio, tx 78232
    - (long: -98.482517, lat: 29.608570)
    
- 78249	salado creek	4100 kingston, san antonio, tx	-98.60637544	29.569293
    - changed to 4100 kingston, san antonio, tx 78218
    - (long: -98.409455, lat: 29.502832)
    
- 78249	salado creek	6011 kingston, san antonio, tx	-98.60416452	29.57100288
    - changed to 6011 kingston, san antonio, tx 78218
    - (long: -98.410942, lat: 29.492495)
    
- 78249	salado creek	6000 kingston, san antonio, tx	-98.60377135	29.57104561
    - changed to 6000 kingston, san antonio, tx 78218
    - (long: -98.409455, lat: 29.502832)
    
- 78249	salado creek	5800 kingston, san antonio, tx	   
    - changed to 5800 kingston, san antonio, tx 78218
    - (long: -98.409455, lat: 29.502832)
    
- 78238	salado creek	4906 wurzbach pkwy, san antonio, tx	-98.61201237	29.47850521
    - changed to 4906 wurzbach pkwy, san antonio, tx 78233
    - (long: -98.38278, lat: 29.545654)
    
- 78260	medio creek	1605 dove canyon, san antonio, tx	-98.46736392	29.68591598
    - changed to 1605 dove canyon, san antonio, tx 78245 
    - (long: -98.718503, lat: 29.419498)
    
- 78249	medio creek	8541 lake vista, san antonio, tx	-98.65468727	29.55706988
    - changed to 8541 lake vista, san antonio, tx 78227
    - (long: -98.650678, lat: 29.394092)
    
Lat and Long and sewershed were wrong:
- 78245	leon creek	2380 grosenbacher rd, san antonio, tx	-98.72656996	29.40217192
    - changed (long: -98.72651, lat: 29.407868)
    - changed sewershed to medio creek

In [74]:
df = acquire.read_data('SAWS_SSOs_2009-2018Mar_UploadData.csv')

In [75]:
df.Root_Cause.value_counts()

STRUCTURAL           1237
GREASE                610
DEBRIS                443
RAIN EVENT            353
CONTRACTOR            164
LIFT STATION          125
OTHER                  61
VANDALISM              48
RAIN EVENT             48
VANDALISM              24
ROOTS                  22
LIFT STATION           15
I/I                    12
CONTRACTOR              8
BY PASS PUMP LEAK       3
Grease                  1
Debris                  1
OTHER                   1
Name: Root_Cause, dtype: int64

In [76]:
df.CAUSE.value_counts()

Grease                                                  956
Debris                                                  479
I/I                                                     465
Debris/Rock                                             254
Structural                                              244
Rags                                                    236
Break In Main                                           164
Contractor                                               91
Break in Main                                            68
Lift Station                                             42
Vandalism                                                38
Lift Station Power Out                                   29
Other                                                    29
Roots                                                    28
Lift Station Pump Out                                    21
Other - Capacity                                          5
Equipment Failure                       

In [77]:
df = pd.read_excel('data/SAWS_SSOs_2009-2018Mar_UploadData.xlsx', sep=', ', keep_default_na=True)

In [78]:
df.columns = map(str.lower, df.columns)

In [79]:
df.rename(index=str, columns={'public notice':'public_notice', 'spill address': 'spill_address_full'}, inplace=True)

In [80]:
df.head()

,sso_id,inspkey,servno,reportdate,spill_address,spill_st_name,total_gal,galsret,gal,spill_start,spill_stop,hrs,cause,comments,actions,watershed,unitid,unitid2,discharge_to,discharge_route,council_district,ferguson,month,year,week,earz_zone,expr1029,pipediam,pipelen,pipetype,instyear,dwndpth,upsdpth,inches_no,rainfall_less3,spill_address_full,sewerassetexp,num_spills_compkey,num_spills_24mos,prevspill_24mos,unittype,assettype,lastclnd,responsetime,responsedttm,public_notice,timeint,root_cause,steps_to_prevent,spill_start_2,spill_stop_2,hrs_2,gal_2,spill_start_3,spill_stop_3,hrs_3,gal_3
0,6582,567722.0,NaN,2019-03-10,3200,THOUSAND OAKS DR,2100,2100.0,2100.0,3/10/2019 1:16:00 PM,3/10/2019 2:40:00 PM,1.400000,Grease,Spill ContainedReturned to SystemArea Cleaned ...,CLEANED MAIN,SALADO CREEK,66918,66917,STREET,None,NaN,172A2,3,2019,11,0.0,NaN,8.0,16.55,PVC,1997.0,NaN,NaN,NaN,NaN,3200 THOUSAND OAKS DR,NaT,1,1.0,NaT,GRAVITY,Sewer Main,NaT,0.450000,2019-03-10 13:43:00,False,24.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
1,6583,567723.0,NaN,2019-03-10,6804,S FLORES ST,80,0.0,80.0,3/10/2019 2:25:00 PM,3/10/2019 3:45:00 PM,1.333333,Grease,Spill ContainedArea Cleaned and Disinfected,CLEANED MAIN,DOS RIOS,24250,24193,STORMDRAIN,None,3.0,251A3,3,2019,11,0.0,NaN,8.0,157.00,PVC,1988.0,NaN,NaN,NaN,NaN,6804 S FLORES,NaT,1,1.0,NaT,GRAVITY,Sewer Main,NaT,1.083333,2019-03-10 15:30:00,False,120.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
2,6581,567714.0,NaN,2019-03-09,215,AUDREY ALENE DR,79,0.0,10.0,3/9/2019 6:00:00 PM,3/9/2019 7:30:00 PM,1.500000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,DOS RIOS,2822,3351,ALLEY,None,1.0,190E4,3,2019,10,0.0,NaN,8.0,350.00,CP,1955.0,NaN,NaN,NaN,NaN,215 Audrey Alene Dr,NaT,1,1.0,NaT,GRAVITY,Sewer Main,NaT,1.000000,2019-03-09 19:00:00,False,24.0,NaN,NaN,03/10/2019 09:36,03/10/2019 10:45,1.15,69.0,NaN,NaN,0.0,0.0
3,6584,567713.0,NaN,2019-03-09,3602,SE MILITARY DR,83,0.0,83.0,3/9/2019 3:37:00 PM,3/9/2019 5:00:00 PM,1.383333,Grease,Spill ContainedArea Cleaned and DisinfectedFlu...,NaN,SALADO CREEK,92804,92805,EASEMENT,None,3.0,252C3,3,2019,10,0.0,NaN,8.0,213.91,PVC,1983.0,NaN,NaN,NaN,NaN,3602 SE MILITARY DR,NaT,1,1.0,NaT,GRAVITY,Sewer Main,NaT,0.550000,2019-03-09 16:10:00,False,120.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
4,6580,567432.0,NaN,2019-03-06,100,PANSY LN,75,0.0,75.0,3/6/2019 9:40:00 AM,3/6/2019 9:55:00 AM,0.250000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,SALADO CREEK,61141,49543,STREET,None,2.0,192A7,3,2019,10,0.0,NaN,12.0,291.90,CP,1952.0,NaN,NaN,NaN,NaN,100 PANSY LN,NaT,2,2.0,2018-12-15,GRAVITY,Sewer Main,NaT,0.000000,2019-03-06 09:40:00,False,3.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0


In [81]:
df.num_spills_24mos[df.num_spills_24mos > 1].value_counts()

2.0     180
3.0      75
4.0      32
5.0      27
6.0      19
7.0      12
8.0       7
9.0       5
10.0      1
11.0      1
12.0      1
13.0      1
Name: num_spills_24mos, dtype: int64

In [82]:
df.full_address_spill.value_counts()

SWISS OAKS 06606                                                                                                                                                                             19
CHAPPIE JAMES WAY 00108                                                                                                                                                                      15
HOLBROOK 00668                                                                                                                                                                               10
HARRY WURZBACH 01427                                                                                                                                                                          9
MOREY RD 05410                                                                                                                                                                                8
CHAPPIE JAMES WAY 00100                 

In [87]:
df[df.gal > 1500000].T

,84,115,646,647,688,2338,2771
sso_id,6499,6466,5942,5943,5888,18,1796
inspkey,NaN,NaN,NaN,NaN,NaN,NaN,NaN
servno,2.51675e+06,2.46636e+06,1.30914e+06,1.31262e+06,1.28896e+06,NaN,NaN
reportdate,2018-10-24 00:00:00,2018-09-20 00:00:00,2016-06-02 00:00:00,2016-06-02 00:00:00,2016-05-18 00:00:00,2010-10-20 00:00:00,2010-01-15 00:00:00
spill_address,108,6606,700,6606,700,8600,300
spill_st_name,CHAPPIE JAMES WAY,SWISS OAKS,HOLBROOK,SWISS OAKS,HOLBROOK,MISSION PKWY,PETALUMA BLVD E
total_gal,2448400,13304550,6026731,2050500,2754790,4442000,3570000
galsret,0,0,0,0,0,NaN,NaN
gal,2.4484e+06,1.33046e+07,6.02673e+06,2.0505e+06,2.75479e+06,4.442e+06,3.57e+06
spill_start,10/24/2018 1:50:00 PM,9/20/2018 9:45:00 AM,6/2/2016 11:09:00 AM,6/2/2016 11:45:00 AM,5/17/2016 6:28:00 PM,10/20/2010 9:00:00 AM,1/15/2010 2:00:00 AM
